In [1]:
import pandas as pd
from comet import download_model, load_from_checkpoint

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-25 14:34:40.421353: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 14:34:40.471736: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 14:34:42.505052: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different n

In [2]:
comet_da_model_path = download_model("wmt21-comet-da")
comet_da_model = load_from_checkpoint(comet_da_model_path)

comet_mqm_model_path = download_model("wmt21-comet-mqm")
comet_mqm_model = load_from_checkpoint(comet_mqm_model_path)

wmt21-comet-da is already in cache.
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../root/.cache/torch/unbabel_comet/wmt21-comet-da/checkpoints/model.ckpt`
Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
wmt21-comet-mqm is already in cache.
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../root/.cache/torch/unbabel_comet/wmt21-comet-mqm/checkpoints/model.ckpt`
Encoder model frozen.


In [3]:
from evaluate import load
cometm = load("comet")

PyTorch version 2.5.1+cu121 available.
TensorFlow version 2.20.0 available.
Fetching 5 files: 100%|████████████████████| 5/5 [00:00<00:00, 62230.03it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.


In [4]:
da = {}
mqm = {}
comet = {}

## Original

In [5]:
file_name = "shalaka_original_outputs.csv"
mode = "original"

In [6]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
src_sentences = df['src']
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:08<00:00,  1.19s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [7]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'original': 0.10275046935270506
Average COMET-MQM for 'original': 0.04243417260133558
Average COMET for 'original': 0.7391001256527724


## Only Punctuations

In [9]:
file_name = "shalaka_only_punct_outputs.csv"
mode = "only_punct"

In [10]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:08<00:00,  1.24s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [11]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'only_punct': 0.09342593503081137
Average COMET-MQM for 'only_punct': 0.04197763744741678
Average COMET for 'only_punct': 0.7425999978074321


## Without Punctuations

In [13]:
file_name = "shalaka_without_punct_outputs.csv"
mode = "without_punct"

In [14]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:07<00:00,  1.13s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [15]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
TOKENIZERS_PARALLEL

In [16]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'without_punct': 0.08840061060187442
Average COMET-MQM for 'without_punct': 0.04155294823080853
Average COMET for 'without_punct': 0.7417267549920965


## Combined

In [17]:
file_name = "shalaka_combined_outputs.csv"
mode = "combined"

In [18]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.05it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [19]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'combined': 0.10059081662998155
Average COMET-MQM for 'combined': 0.04204615822958725
Average COMET for 'combined': 0.743648981054624


## Combined - LR & Epochs Changed

In [21]:
file_name = "shalaka_combined_le_outputs.csv"
mode = "combined_le"

In [22]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:08<00:00,  1.16s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [23]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'combined_le': 0.09786708270096117
Average COMET-MQM for 'combined_le': 0.04188514849240029
Average COMET for 'combined_le': 0.7442724417757105


## Combined - LR & Epochs & Dataset Changed

In [25]:
file_name = "shalaka_combined_led_outputs.csv"
mode = "combined_led"

In [26]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.03it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [27]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'combined_led': 0.0951224053968434
Average COMET-MQM for 'combined_led': 0.04170900252130297
Average COMET for 'combined_led': 0.743326163954205


## GPT-5

In [29]:
file_name = "shalaka_gpt_outputs.csv"
mode = "gpt"

In [30]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:07<00:00,  1.09s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [31]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'gpt': 0.09327893148831747
Average COMET-MQM for 'gpt': 0.04157844986076708
Average COMET for 'gpt': 0.7419534800229249


## BERT

In [33]:
file_name = "approach1_bert_outputs.csv"
mode = "bert"

In [34]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.08it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [35]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'bert': 0.11674263634443006
Average COMET-MQM for 'bert': 0.04321766975853178
Average COMET for 'bert': 0.7590071360270182


## MPNet

In [37]:
file_name = "approach1_mpnet_outputs.csv"
mode = "mpnet"

In [38]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:08<00:00,  1.17s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [39]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'mpnet': 0.11757941007683123
Average COMET-MQM for 'mpnet': 0.04343687335926073
Average COMET for 'mpnet': 0.7598996581854643


## T5

In [41]:
file_name = "approach1_t5_outputs.csv"
mode = "t5"

In [42]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df['gemini'].tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.07it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [43]:
comet_score = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 't5': 0.12037710893761229
Average COMET-MQM for 't5': 0.04382241741512661
Average COMET for 't5': 0.7573415614940502


In [45]:
for key, val in da.items():
    print(f"{key} : {val:.4f}")

original : 0.1028
only_punct : 0.0934
without_punct : 0.0884
combined : 0.1006
combined_le : 0.0979
combined_led : 0.0951
gpt : 0.0933
bert : 0.1167
mpnet : 0.1176
t5 : 0.1204


In [46]:
for key, val in mqm.items():
    print(f"{key} : {val:.4f}")

original : 0.0424
only_punct : 0.0420
without_punct : 0.0416
combined : 0.0420
combined_le : 0.0419
combined_led : 0.0417
gpt : 0.0416
bert : 0.0432
mpnet : 0.0434
t5 : 0.0438


In [47]:
for key, val in comet.items():
    print(f"{key} : {val:.4f}")

original : 0.7391
only_punct : 0.7426
without_punct : 0.7417
combined : 0.7436
combined_le : 0.7443
combined_led : 0.7433
gpt : 0.7420
bert : 0.7590
mpnet : 0.7599
t5 : 0.7573
